In [1]:
# utils
import pandas as pd
import os
import numpy as np
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import ast
import re
import pickle

# evaluation 
from evaluate import load

In [2]:
# open files in generations directory
def open_files(directory):
    files = os.listdir(directory)
    files.sort()
    return files

In [3]:
models_generations = {
    'it': {},
}

files = open_files('generations')
for file in files:
    model = re.sub(r"^fine-tuned-|-decoding-gen\d+.csv", "", file)
    gen = re.search(r"gen\d+", file).group()

    model_generations = pd.read_csv('generations/' + file)
    # convert string to list
    model_generations['actual'] = model_generations['actual'].apply(ast.literal_eval)
    
    # if containt -en-
    if '-it-' in file:
        if model not in models_generations['it']:
            models_generations['it'][model] = {}
        models_generations['it'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: it")

model: LLaMAntino-3-ANITA-8B-Inst-DPO-ITA-it - gen: gen0, lang: it
model: LLaMAntino-3-ANITA-8B-Inst-DPO-ITA-it - gen: gen1, lang: it
model: LLaMAntino-3-ANITA-8B-Inst-DPO-ITA-it - gen: gen2, lang: it
model: LLaMAntino-3-ANITA-8B-Inst-DPO-ITA-it-sga - gen: gen0, lang: it
model: LLaMAntino-3-ANITA-8B-Inst-DPO-ITA-it-sga - gen: gen1, lang: it
model: LLaMAntino-3-ANITA-8B-Inst-DPO-ITA-it-sga - gen: gen2, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen0, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen1, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen2, lang: it
model: Llama-3.1-8B-Instruct-it-sga - gen: gen0, lang: it
model: Llama-3.1-8B-Instruct-it-sga - gen: gen1, lang: it
model: Llama-3.1-8B-Instruct-it-sga - gen: gen2, lang: it
model: Minerva-7B-instruct-v1.0-it - gen: gen0, lang: it
model: Minerva-7B-instruct-v1.0-it - gen: gen1, lang: it
model: Minerva-7B-instruct-v1.0-it - gen: gen2, lang: it
model: Minerva-7B-instruct-v1.0-it-sga - gen: gen0, lang: it
model: Minerva-7B-

## Valutazione automatica

### <a href="https://huggingface.co/spaces/evaluate-metric/meteor">METEOR</a>

In [9]:
meteor = load('meteor')

Using the latest cached version of the module from C:\Users\OliverioM\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--meteor\e7ed321a1b44c34fa4679192809db2cee7e3bd4bba0fe8b76061d807706c2374 (last modified on Thu Oct 10 16:06:59 2024) since it couldn't be found locally at evaluate-metric--meteor, or remotely on the Hugging Face Hub.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\OliverioM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\OliverioM\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\OliverioM\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
meteor_scores = {'it': {}}

In [11]:
# open chrf_scores-exp3.pkl
with open('meteor_scores-exp4.pkl', 'rb') as f:
    meteor_scores = pickle.load(f)

In [12]:
list(meteor_scores['it'].keys())

['LLaMAntino-3-ANITA-8B-Inst-DPO-ITA-it',
 'LLaMAntino-3-ANITA-8B-Inst-DPO-ITA-it-sga',
 'Llama-3.1-8B-Instruct-it',
 'Llama-3.1-8B-Instruct-it-sga',
 'Minerva-7B-instruct-v1.0-it',
 'Mistral-Nemo-Instruct-2407-it',
 'Mistral-Nemo-Instruct-2407-it-sga',
 'Qwen2.5-7B-Instruct-it',
 'Qwen2.5-7B-Instruct-it-sga']

In [13]:
for lang in models_generations:
    for model in models_generations[lang]:
        if model not in meteor_scores[lang]:
            model_meteor_scores = []
            print(f'Language: {lang}, Model: {model}')
            
            for gen in sorted(models_generations[lang][model]):  # Ensure order (gen0, gen1, gen2)
                references = models_generations[lang][model][gen]['actual']
                predictions = models_generations[lang][model][gen]['prediction']

                gen_meteor_scores = []
                for i in tqdm(range(len(references)), desc="Calcolo punteggi METEOR"):
                    results = meteor.compute(predictions=[predictions[i]], references=[references[i]])
                    gen_meteor_scores.append(results["meteor"])

                model_meteor_scores.append(gen_meteor_scores)  # Store BLEU for each generation
            
            # Store the three METEOR scores instead of their mean
            meteor_scores[lang][model] = model_meteor_scores
            print(f'BLEU scores for {model} ({lang}): {model_meteor_scores}')
            print(f'Average: {np.mean(model_meteor_scores)}')
            print()

Language: it, Model: Minerva-7B-instruct-v1.0-it-sga


Calcolo punteggi METEOR:   0%|          | 0/710 [00:00<?, ?it/s]Exception ignored in: <function ZipFile.__del__ at 0x0000022AB10B74C0>
Traceback (most recent call last):
  File "c:\Users\OliverioM\.pyenv\pyenv-win\versions\3.11.0\Lib\zipfile.py", line 1870, in __del__
    self.close()
  File "c:\Users\OliverioM\.pyenv\pyenv-win\versions\3.11.0\Lib\zipfile.py", line 1892, in close
    self._fpclose(fp)
  File "c:\Users\OliverioM\.pyenv\pyenv-win\versions\3.11.0\Lib\zipfile.py", line 1989, in _fpclose
    assert self._fileRefCnt > 0
AssertionError: 
Calcolo punteggi METEOR: 100%|██████████| 710/710 [00:07<00:00, 95.34it/s] 

BLEU scores for Minerva-7B-instruct-v1.0-it-sga (it): [[0.42763049631384936, 0.4653846153846154, 0.2765132874015748, 0.6957463100361734, 0.37431258729050276, 0.5513028169014084, 0.5389984357965586, 0.4277966351829989, 0.7372078981513599, 0.8069387755102042, 0.46477462925470875, 0.7491586481490258, 0.4481904729319385, 0.4435444790584977, 0.46053874766280767, 0.7395226521610536, 0.5552685950413223, 0.5018506680556543, 0.7240592814133041, 0.7028919566761364, 0.5272452007801218, 0.52991452991453, 0.36579190305898385, 0.6343290441176471, 0.6572745689833382, 0.9316823437193809, 0.7178240995143953, 0.4872869318181818, 0.5010893246187363, 0.42270742358078606, 0.8581560283687943, 0.703125, 0.7229980051296665, 0.8680555555555556, 0.4842781557067271, 0.7507738807931548, 0.6380670611439841, 0.4768822286476868, 0.6327927671440878, 0.4193957683207023, 0.6030932552393153, 0.3445692883895132, 0.48218029350104813, 0.2670737750313348, 0.524459766347482, 0.7193833415719229, 0.6437858609254092, 0.50179445

In [14]:
# Save meteor scores
with open('meteor_scores-exp4.pkl', 'wb') as f:
    pickle.dump(meteor_scores, f)